In [2]:
import xarray as xr
import numpy as np
import glob
import os

In [3]:
phase='calval_phase' # calval_phase or science_phase
simu='eNATL60-BLB002' # or eNATL60-BLBT02
data='karin' # or nadir

In [4]:
#parameters for the region to consider
reg='NANFL'
latmin=30
latmax=40
lonmin=-55+360
lonmax=-40+360


In [5]:
tdir='/work/ALT/swot/aval/wisa/tmp/inland_cleaning/swot_simulator_'+phase+'/'+simu+'-SSH-1h/'+data
files=glob.glob(tdir+'/*/*nc')

In [6]:
def check_data_region(file,latmin,latmax,lonmin,lonmax):
    ds=xr.open_dataset(file)
    latfile=ds.latitude
    lonfile=ds.longitude
    ind_region=np.where((latfile < latmax) & (latfile > latmin) & 
        (lonfile < lonmax) & (lonfile > lonmin))
    if len(ind_region[0]) > 0:
        check=1
    else:
        check=0
    return check,ind_region
    

In [7]:
def check_data_nonan(file,ind_region):
    ds=xr.open_dataset(file)
    latfile=ds.latitude
    lonfile=ds.longitude
    ssh=ds.ssh_karin    
    numlines_region=ind_region[0]
    numlines_region_unique=np.unique(numlines_region)
    for k in np.arange(len(numlines_region_unique)):
        isdata=np.where(np.isnan(ssh[numlines_region_unique[k],:])==False)
        if len(isdata[0])>0:
            check=1
        else:
            check=0
        return check,isdata


In [8]:
for f in np.arange(len(files)):
    file=files[f]
    nfile=file.split('/')[-1]
    odir='/work/ALT/odatis/eNATL60/alberta/SWOT-sim/'+reg+'/'+phase+'/'+data+'/'
    filename=odir+nfile
    if not os.path.exists(filename):
        check, ind_region = check_data_region(file,latmin,latmax,lonmin,lonmax)
        if check == 1:
            check2,isdata = check_data_nonan(file,ind_region)
            if check2 == 1:
                print('extract data for file '+file)
                ds=xr.open_dataset(file)
                ind_region_unique=np.unique(ind_region[0])
                ds_regnonan=ds.isel(num_lines=ind_region_unique[isdata[0]])
                if not os.path.exists(odir):
                    os.makedirs(odir)
                ds_regnonan.to_netcdf(path=filename,mode='w')

extract data for file /work/ALT/swot/aval/wisa/tmp/inland_cleaning/swot_simulator_calval_phase/eNATL60-BLB002-SSH-1h/karin/2010/SWOT_L2_LR_SSH_Basic_341_011_20100603T042122_20100603T051228_DG10_01.nc
extract data for file /work/ALT/swot/aval/wisa/tmp/inland_cleaning/swot_simulator_calval_phase/eNATL60-BLB002-SSH-1h/karin/2010/SWOT_L2_LR_SSH_Basic_374_024_20100706T101600_20100706T110705_DG10_01.nc
extract data for file /work/ALT/swot/aval/wisa/tmp/inland_cleaning/swot_simulator_calval_phase/eNATL60-BLB002-SSH-1h/karin/2010/SWOT_L2_LR_SSH_Basic_277_011_20100331T142145_20100331T151251_DG10_01.nc
extract data for file /work/ALT/swot/aval/wisa/tmp/inland_cleaning/swot_simulator_calval_phase/eNATL60-BLB002-SSH-1h/karin/2010/SWOT_L2_LR_SSH_Basic_213_024_20100127T112622_20100127T121727_DG10_01.nc
extract data for file /work/ALT/swot/aval/wisa/tmp/inland_cleaning/swot_simulator_calval_phase/eNATL60-BLB002-SSH-1h/karin/2010/SWOT_L2_LR_SSH_Basic_398_024_20100730T063051_20100730T072157_DG10_01.nc
